In [150]:
#loading libraries
import torch as torch
import torch.nn as nn
import pandas as pd
import numpy as np
from torch.autograd import Variable
from sklearn.metrics import accuracy_score
import torch.nn.functional as F
#data preprocessing

datatrain=pd.read_csv('Iris.csv')
datatrain.loc[datatrain['Species']=='Iris-setosa', 'Species']=0
datatrain.loc[datatrain['Species']=='Iris-versicolor', 'Species']=1
datatrain.loc[datatrain['Species']=='Iris-virginica', 'Species']=2
datatrain = datatrain.apply(pd.to_numeric)
datatrain=np.array(datatrain)



#split x and y (feature and target)
X = datatrain[:,1:5]
y = datatrain[:,5]

y=pd.get_dummies(y)
y=np.array(y)
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y)

X_train=torch.from_numpy(X_train)
y_train=torch.from_numpy(y_train)
X_test=torch.from_numpy(X_test)
y_test=torch.from_numpy(y_test)

# Loss Function

In [151]:
#Custom loss function that penalizes large weights
class MyLoss(torch.nn.Module):
    
    def __init__(self,eps1,eps2,beta):
        super(MyLoss,self).__init__()
        self.eps1=eps1
        self.eps2=eps2
        self.beta=beta
        
    def forward(self,target,cat_label):
        ploss=self.eps1*(torch.sum((self.beta*(net.fc1.weight.clone()**2))/(1+(self.beta*(net.fc1.weight.clone()**2))))+torch.sum((self.beta*(net.fc2.weight.clone()**2))/(1+(self.beta*(net.fc2.weight.clone()**2)))))
        +self.eps2*torch.sum(net.fc1.weight.clone()**2)+self.eps2*torch.sum(net.fc2.weight.clone()**2)
        celoss=-(torch.sum(cat_label * torch.log(target) + (1 - cat_label) * torch.log(1-target)))
        totloss=celoss+ploss
        
        return totloss

# Model with Custom Layer

In [152]:
#class to drop connections between two neurons
class MaskedLinear(nn.Linear):
    def __init__(self, in_features, out_features, bias=True):
        super(MaskedLinear, self).__init__(in_features, out_features, bias)
        self.mask_flag = False
    
    def set_mask(self, mask):
        self.register_buffer('mask', mask)
        mask_var = self.get_mask()
        self.weight.data = self.weight.data*mask_var.data
        self.mask_flag = True
    
    def get_mask(self):
        # print(self.mask_flag)
        return Variable(self.mask, requires_grad=False)
    
    def forward(self, x):
        if self.mask_flag == True:
            mask_var = self.get_mask()
            weight = self.weight * mask_var
            return F.linear(x, weight, self.bias)
        else:
            return F.linear(x, self.weight, self.bias)

In [153]:
#network definition
class Net(nn.Module):
    def __init__(self,input_size,hidden_size,num_classes):
        super(Net, self).__init__()
        self.fc1 = MaskedLinear(input_size, hidden_size,bias=1) 
        self.tanh = nn.Tanh()
        self.fc2 = MaskedLinear(hidden_size, num_classes,bias=1)
        self.sigm=nn.Sigmoid()
    
    
    def forward(self,x):
        x=self.fc1(x)
        x=self.tanh(x)
        out=self.fc2(x)
        out=self.sigm(out)
        
        return out
    def store_neuron_outputs(self,x):
        x=self.fc1(x)
        x=self.tanh(x)
        
        return x
        
    def set_masks(self, masks):
        self.fc1.set_mask(masks[0])
        self.fc2.set_mask(masks[1])

In [154]:
net =Net(4,1,3)
criterion = MyLoss(0.01,0.01,10)
#criterion=nn.NLLLoss()
optimizer = torch.optim.RMSprop(net.parameters(), lr=0.001)
X_tr = Variable(X_train).float()
Y_hot = Variable(y_train).float()
Y_cat=np.where(y_train==1)[1]
Y_cat = Variable(torch.from_numpy(Y_cat)).long()

# Pruning Algorithm

In [162]:
#function that trains a network and prunes connections with least importance until it does not improve accuracy
def prune_network(net,no_of_neurons):
    maskfc1=torch.ones_like(model.fc1.weight.data)
    maskfc2=torch.ones_like(model.fc2.weight.data)
    masks=[]
    masks.append(maskfc1)
    masks.append(maskfc2)
    while(1):
        removed=False
        for m in range(no_neurons):
            for l in range(net.fc1.weight.data.shape[1]):
                wml=net.fc1.weight.clone()[m,:][l]
                if(torch.max(torch.abs(net.fc1.weight.clone()[m,:][0]*net.fc2.weight.clone()[:,m]))<4*neta2):
                    masks=remove(net,masks,"fc1",m,l)
                    removed=True
            for p in range(net.fc2.weight.data.shape[0]):
                vpm=net.fc2.weight.clone()[:,m][p]
                if(torch.abs(vpm)<4*neta2):
                    masks=remove(net,masks,"fc2",m,p)
                    removed=True
        if(removed==True):
            net.set_masks(masks)
            train(net)
            if(accuracy<prev_accuracy):
                continue
            else:
                break
        else:
            WML=[]
            for m in range(no_neurons):
                for l in range(net.fc1.weight.data.shape[1]):
                    wml=net.fc1.weight.clone()[m,:][l]
                    wml=torch.max(torch.abs(net.fc1.weight.clone()[m,:][0]*net.fc2.weight.clone()[:,m]))
                    WML.append(wml)
            remove(min(WML))
        net.set_masks(masks)
        train(net)
        if(accuracy<prev_accuracy):
            continue
        else:
            break

In [163]:
#remove specified connections
def remove(model,prev_masks,layer,m,l):
    curr_masks=[]
    mask1=torch.ones_like(model.fc1.weight.data)
    mask2=torch.ones_like(model.fc2.weight.data)
    if(layer=="fc1"):
        mask1[m,:][l]=0
    elif(layer=="fc2"):
        mask2[:,m][l]=0
    curr_masks.append(prev_masks[0]*mask1)
    curr_masks.append(prev_masks[1]*mask2)
    
    return curr_masks

In [164]:

#trains the model 
def train(model,no_epochs,loss_fn,optmizer):
    for i in range(no_of_epochs):
        optimizer.zero_grad()
    
        out =model(X)
        loss = loss_fn(out,Y_hot)
        loss.backward()
    
        optimizer.step()
        print(loss.data[0])